In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pdb

from data_prep import prepareData

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Data prep

In [2]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 30

input_file = 'iwslt-vi-en-processed/train.vi'
target_file = 'iwslt-vi-en-processed/train.en'
input_lang, target_lang, pairs = prepareData(input_file, target_file, 'vi', 'eng', size=50000)

Reading lines...
['khoa hoc ang sau mot tieu e ve khi hau', 'rachel pike the science behind a climate headline']
Read 50000 sentence pairs
Trimmed to 37026 sentence pairs
Counting words...
['khoa hoc ang sau mot tieu e ve khi hau', 'rachel pike the science behind a climate headline']
Counted words:
vi 5551
eng 19344


In [ ]:
print(pairs[4][0])
print(pairs[4][1])

In [ ]:
import pickle
pickle.dump(pairs, open("preprocessed_data_no_elmo/iwslt-vi-eng/preprocessed_no_indices_pairs_test", "wb"))
pickle.dump(input_lang, open("preprocessed_data_no_elmo/iwslt-zh-eng/preprocessed_no_elmo_zhlang", "wb"))
pickle.dump(target_lang, open("preprocessed_data_no_elmo/iwslt-zh-eng/preprocessed_no_elmo_englang", "wb"))


#### Encoder and Decoder

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embed = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embed, hidden)
        # output and hidden are the same vectors
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        embed = self.embedding(input).view(1, 1, -1)
        embed = F.relu(embed)
        output, hidden = self.gru(embed, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [37]:
class CorrectDecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(CorrectDecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(2*hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_hidden):
        embed = self.embedding(input).view(1, 1, -1)
        embed = F.relu(embed)
        embed_plus_encoded = torch.cat((embed, encoder_hidden), 2)
        output, hidden = self.gru(embed_plus_encoded, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Training
#### Preparing Training Data

In [23]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(target_lang, pair[1])
    return (input_tensor, target_tensor)




# def indexesFromSentence(lang, sentence):
#     words = sentence.split(' ')
#     indices = []
#     for word in words:
#         if lang.word2index.get(word) is not None:
#             indices.append(lang.word2index[word])
#         else:
#             indices.append(1) # UNK_INDEX
#     return indices


# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


# def tensorsFromPair(pair, input_lang, target_lang):
#     input_tensor = tensorFromSentence(input_lang, pair[0])
#     target_tensor = tensorFromSentence(target_lang, pair[1])
#     return (input_tensor, target_tensor)



### Training the Model

In [27]:
teacher_forcing_ratio = 0.5


# example of input_tensor: [2, 43, 23, 9, 19, 4]. Indexed on our vocabulary. 
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    # iterate GRU over words --> final hidden state is representation of source sentence. 
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = decoder.initHidden()

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [6]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

In [17]:
# def load_cpickle_gc(dirlink):
#     # https://stackoverflow.com/questions/26860051/how-to-reduce-the-time-taken-to-load-a-pickle-file-in-python
#     output = open(dirlink, 'rb')

#     # disable garbage collector
#     gc.disable()

#     mydict = pickle.load(output)

#     # enable garbage collector again
#     gc.enable()
#     output.close()
#     return mydict

# def trainIters(encoder, decoder, n_iters,n_epochs,  lang1, lang2,  print_every=1000, plot_every=100, learning_rate=0.001):
#     """
#     lang1 is the Lang object for language 1 
#     Lang2 is the Lang object for language 2
#     """
#     pairs = load_cpickle_gc("preprocessed_data_no_elmo/iwslt-"+lang1.name+"-"+lang2.name+"/preprocessed_no_indices_pairs_train")
#     validation_pairs = load_cpickle_gc("preprocessed_data_no_elmo/iwslt-"+lang1.name+"-"+lang2.name+"/preprocessed_no_indices_pairs_train")
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every
#     encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = torch.optim.SGD(decoder.parameters(), lr=learning_rate)
#     for i in range(n_epochs):
#         training_pairs = [tensorsFromPair(random.choice(pairs))
#                           for i in range(n_iters)]
#         criterion = nn.NLLLoss()
#         # framing it as a categorical loss function. 
#         for iter in range(1, n_iters + 1):
#             training_pair = training_pairs[iter - 1] 
#             d_input_tensor = training_pair[0]
#             d_target_tensor = training_pair[1]
#             input_tensor = tensorFromSentence(lang1, d_input_tensor)
#             target_tensor = tensorFromSentence(lang2, d_target_tensor)
#             loss = train(input_tensor, target_tensor, encoder,
#                          decoder, encoder_optimizer, decoder_optimizer, criterion, MAX_LENGTH_VI_EN)
#             print_loss_total += loss
#             plot_loss_total += loss

#             if iter % print_every == 0:
#                 print_loss_avg = print_loss_total / print_every
#                 print_loss_total = 0
#                 print('TRAIN SCORE %s (%d %d%%) %.4f' % (timeSince(start, iter / n_epochs),
#                                              iter, iter / n_epochs * 100, print_loss_avg))
#                 val_loss = test_model(encoder, decoder,search, validation_pairs, lang1, max_length=MAX_LENGTH)
#                 # retursn teh bleu score
#                 print("VALIDATION BLEU SCORE: "+str(val_loss))

#             if iter % plot_every == 0:
#                 plot_loss_avg = plot_loss_total / plot_every
#                 plot_losses.append(plot_loss_avg)
#                 plot_loss_total = 0

#     showPlot(plot_losses)

In [18]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [8]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [38]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = CorrectDecoderRNN(target_lang.n_words, hidden_size).to(device)

num_iters = 10000
trainIters(encoder1, decoder1, num_iters, print_every=100)

0m 30s (- 50m 4s) (100 1%) 5.7816
1m 2s (- 50m 55s) (200 2%) 5.4874
1m 30s (- 48m 52s) (300 3%) 4.8540
2m 1s (- 48m 41s) (400 4%) 5.1465
2m 37s (- 49m 50s) (500 5%) 5.3618


KeyboardInterrupt: 

In [ ]:
# torch.save(encoder1.state_dict(), "encoder1_40000")
# torch.save(decoder1.state_dict(), "decoder1_40000")

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
encoder1.load_state_dict(torch.load("encoder1_40000", map_location='cpu'))
decoder1 = DecoderRNN(target_lang.n_words, hidden_size).to(device)
decoder1.load_state_dict(torch.load("decoder1_40000", map_location='cpu'))

evaluateRandomly(encoder1, decoder1, n = 10, strategy='beam', k = 8)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10, strategy="greedy", k = None):
    """
    Randomly select a sentence from the input dataset and try to produce its translation.
    """    
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = generate_translation(encoder, decoder, pair[0], search=strategy, k = k)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
def generate_translation(encoder, decoder, sentence, max_length=MAX_LENGTH, search="greedy", k = None):
    """ 
    @param max_length: the max # of words that the decoder can return
    @returns decoded_words: a list of words in target language
    """    
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        
        # encode the source sentence
        encoder_hidden = encoder.initHidden()
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)

        # start decoding
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = []
        
        if search == 'greedy':
            decoded_words = greedy_search(decoder, decoder_input, decoder_hidden, max_length)
        elif search == 'beam':
            if k == None:
                k = 2
            decoded_words = beam_search(decoder, decoder_input, decoder_hidden, max_length, k)  

        return decoded_words

In [ ]:
def greedy_search(decoder, decoder_input, hidden, max_length):
    translation = []
    for i in range(max_length):
        next_word_softmax, hidden = decoder(decoder_input, hidden)
        best_idx = torch.max(next_word_softmax, 1)[1].squeeze().item()

        # convert idx to word
        best_word = target_lang.index2word[best_idx]
        translation.append(best_word)
        decoder_input = torch.tensor([[best_idx]], device=device)
        
        if best_word == 'EOS':
            break
    return translation


def beam_search(decoder, decoder_input, hidden, max_length, k):
    
    candidates = [(decoder_input, 0, hidden)]
    potential_candidates = []
    completed_translations = []

    # put a cap on the length of generated sentences
    for m in range(max_length):
        for c in candidates:
            # unpack the tuple
            c_sequence = c[0]
            c_score = c[1]
            c_hidden = c[2]
            # EOS token
            if c_sequence[-1] == 1:
                completed_translations.append((c_sequence, c_score))
                k = k - 1
            else:
                next_word_probs, hidden = decoder(c_sequence[-1], c_hidden)
                # in the worst-case, one sequence will have the highest k probabilities
                # so to save computation, only grab the k highest_probability from each candidate sequence
                top_probs, top_idx = torch.topk(next_word_probs, k)
                for i in range(len(top_probs[0])):
                    word = torch.from_numpy(np.array([top_idx[0][i]]).reshape(1, 1)).to(device)
                    new_score = c_score + top_probs[0][i]
                    potential_candidates.append((torch.cat((c_sequence, word)).to(device), new_score, hidden))

        candidates = sorted(potential_candidates, key= lambda x: x[1], reverse=True)[0:k] 
        potential_candidates = []

    completed = completed_translations + candidates
    completed = sorted(completed, key= lambda x: x[1], reverse=True)[0] 
    final_translation = []
    for x in completed[0]:
        final_translation.append(target_lang.index2word[x.squeeze().item()])
    return final_translation

In [ ]:
a = [1, 2, 3]
for x in a: 
    x = 4
print(a)

In [ ]:
a = torch.randn(4, 1)
b = torch.from_numpy(np.array([43]).reshape(1, 1))
torch.cat((a, b))

In [ ]:
def evaluate(encoder, decoder, sentence, search="greedy", max_length=MAX_LENGTH):
    """
    Function that generate translation.
    First, feed the source sentence into the encoder and obtain the hidden states from encoder.
    Secondly, feed the hidden states into the decoder and unfold the outputs from the decoder.
    Lastly, for each outputs from the decoder, collect the corresponding words in the target language's vocabulary.
    And collect the attention for each output words.
    @param encoder: the encoder network
    @param decoder: the decoder network
    @param sentence: string, a sentence in source language to be translated
    @param max_length: the max # of words that the decoder can return
    @output decoded_words: a list of words in target language
    @output decoder_attentions: a list of vector, each of which sums up to 1.0
    """    
    # process input sentence
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        # encode the source lanugage
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        # decode the context vector
        decoder_hidden = encoder_hidden # decoder starts from the last encoding sentence
        # output of this function
        decoder_attentions = torch.zeros(max_length, max_length)
        
        if search == 'greedy':
            decoded_words = greedy_search(decoder, decoder_input, decoder_hidden, max_length)
        elif search == 'beam':
            decoded_words = beam_search(decoder, decoder_input, decoder_hidden, max_length)  
        return decoded_words

In [ ]:
import sacrebleu
def calculate_bleu(predictions, labels):
	"""
	Only pass a list of strings 
	"""
	# tthis is ony with n_gram = 4

	bleu = sacrebleu.raw_corpus_bleu(predictions, [labels], .01).score
	return bleu

In [ ]:
model_e = EncoderRNN(5551, 256)
model_e.load_state_dict(torch.load("encoder1_40000", map_location='cpu'))
model_e.eval()

In [ ]:
model_d = DecoderRNN(19344, 256)
model_d.load_state_dict(torch.load("decoder1_40000", map_location='cpu'))
model_d.eval()

In [ ]:
MAX_LENGTH = 100
def test_model(encoder, decoder,search, test_pairs, lang1,max_length=MAX_LENGTH):
    # for test, you only need the lang1 words to be tokenized,
    # lang2 words is the true labels
    encoder_inputs = [pair[0] for pair in test_pairs]
    true_labels = [pair[1] for pair in test_pairs]
    translated_predictions = []
    for i in range(len(encoder_inputs)): 
        if i% 100== 0:
            print(i)
        e_input = encoder_inputs[i]
        decoded_words = evaluate(encoder, decoder, e_input, max_length=MAX_LENGTH)
        translated_predictions.append(" ".join(decoded_words))
    return calculate_bleu(translated_predictions, true_labels)
    

TODO: 
    
Yikes, teh decoder isn't' preforming very well. 

In [ ]:
test_pairs = pickle.load(open("preprocessed_data_no_elmo/iwslt-vi-eng/preprocessed_no_indices_pairs_test", "rb"))

In [ ]:
input_lang = pickle.load(open("preprocessed_data_no_elmo/iwslt-vi-eng/preprocessed_no_elmo_vilang", "rb"))

In [ ]:
test_model(model_e, model_d, "greedy", test_pairs, input_lang )